In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-mar-2021/train.csv
/kaggle/input/tabular-playground-series-mar-2021/test.csv


In [2]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

pd.set_option('display.max_columns',None)

import optuna
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.metrics import roc_auc_score

import xgboost as xgb
from xgboost import XGBClassifier

In [3]:
train=pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/train.csv')
test=pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/test.csv')
sub=pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv')

In [4]:
cols_to_drop=['id','cat10','cat5','cat7','cat8']
train.drop(columns=cols_to_drop,inplace=True)
test.drop(columns=cols_to_drop,inplace=True)

In [5]:
cat_columns=['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat6', 'cat9', 'cat11',
       'cat12', 'cat13', 'cat14', 'cat15', 'cat16', 'cat17', 'cat18']
cont_columns=['cont0',
       'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7', 'cont8',
       'cont9', 'cont10']
all_features=cat_columns+cont_columns

In [6]:
oe=OrdinalEncoder()
train[cat_columns]=oe.fit_transform(train[cat_columns])
test[cat_columns]=oe.transform(test[cat_columns])

In [7]:
sc=StandardScaler()
train[cont_columns]=sc.fit_transform(train[cont_columns])
test[cont_columns]=sc.transform(test[cont_columns])

In [8]:
data=train.drop(columns=['target'])
target=train['target']

# Optuna

In [9]:
# def objective(trial, data=data, target=target):
#     train_x,test_x,train_y,test_y = train_test_split(data, target,test_size=0.2, random_state=101)
#     params = {
#         'eval_metric' : 'auc',
#         'booster' : 'gbtree',
#         'tree_method' : 'gpu_hist' , 
#         'use_label_encoder' : False , 
#         'lambda' : trial.suggest_loguniform('lambda' , 1e-5 , 1.0),
#         'alpha' : trial.suggest_loguniform('alpha' , 1e-5 , 1.0),
#         'colsample_bytree' : trial.suggest_uniform('colsample_bytree' , 0 , 1.0),
#         'subsample' : trial.suggest_uniform('subsample' , 0 , 1.0),
#         'learning_rate' : trial.suggest_uniform('learning_rate' , 0 , 0.02),
#         'n_estimators' : trial.suggest_int('n_estimators' , 1 , 9999),
#         'max_depth' : trial.suggest_int('max_depth' , 1 , 20),
#         'random_state' : trial.suggest_categorical('random_state' , [0,42,2021]),
#         'min_child_weight' : trial.suggest_int('min_child_weight' , 1 , 300),
#         'gamma' : trial.suggest_loguniform('gamma' , 1e-5 , 1.0)
#     }
        
#     model=xgb.XGBClassifier(**params)
#     model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=222,verbose=False)
#     preds=model.predict_proba(test_x)[:,1]
#     auc=roc_auc_score(test_y,preds)
#     return auc

In [10]:
# study=optuna.create_study(direction='maximize',study_name='xgbclassifier')
# study.optimize(objective,n_trials=60)
# print('number of the finished trials:' , len(study.trials))
# print('the parametors of best trial:' , study.best_trial.params)
# print('best value:' , study.best_value)

In [11]:
# optuna.visualization.plot_optimization_history(study)

In [12]:
# optuna.visualization.plot_param_importance(study)

In [13]:
# print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

In [14]:
# params={'lambda': 0.014576256860713442, 'alpha': 0.26389513426946043, 'colsample_bytree': 0.5165904538770105, 'subsample': 0.737564575608887, 'learning_rate': 0.010786623160708754, 'n_estimators': 8790, 'max_depth': 17, 'random_state': 0, 'min_child_weight': 30, 'gamma': 7.272747952404219e-05}

In [15]:
# params

In [16]:
# params['eval_metric'] = 'auc'
# params['booster'] = 'gbtree'
# params['tree_method'] = 'gpu_hist'
# params['use_label_encoder'] = False

In [17]:
# params={'lambda': 0.014576256860713442,
#  'alpha': 0.26389513426946043,
#  'colsample_bytree': 0.5165904538770105,
#  'subsample': 0.737564575608887,
#  'learning_rate': 0.010786623160708754,
#  'n_estimators': 8790,
#  'max_depth': 17,
#  'random_state': 0,
#  'min_child_weight': 30,
#  'gamma': 7.272747952404219e-05,
#  'eval_metric': 'auc',
#  'booster': 'gbtree',
#  'tree_method': 'gpu_hist',
#  'use_label_encoder': False}

In [18]:
# xgb_model = XGBClassifier(
#         random_state=0,
#         n_estimators=8790,
#         verbosity=1,
#         eval_metric="auc",
#         tree_method="gpu_hist",
#         gpu_id=0,
#         booster= 'gbtree',
#         alpha=0.26389513426946043,
#         colsample_bytree=0.5165904538770105,
#         gamma=7.272747952404219e-05,
#         reg_lambda=0.014576256860713442,
#         learning_rate=0.010786623160708754,
#         max_bin=338,
#         max_depth=17,
#         min_child_weight=30,
#         subsample=0.737564575608887,
#     )

In [19]:
preds=np.zeros(test.shape[0])
oof_predictions=np.zeros(len(data))
kf=StratifiedKFold(n_splits=20,random_state=42,shuffle=True)
roc=[]
n=0
for trn_idx,val_idx in kf.split(data,target):
    train_x = data.iloc[trn_idx]
    train_y = target.iloc[trn_idx]
    val_x = data.iloc[val_idx]
    val_y = target.iloc[val_idx]
    
    xgb_model=XGBClassifier(
        random_state=0,
        n_estimators=8790,
        verbosity=1,
        eval_metric="auc",
        tree_method="gpu_hist",
        gpu_id=0,
        booster= 'gbtree',
        alpha=0.26389513426946043,
        colsample_bytree=0.5165904538770105,
        gamma=7.272747952404219e-05,
        reg_lambda=0.014576256860713442,
        learning_rate=0.010786623160708754,
        max_bin=338,
        max_depth=17,
        min_child_weight=30,
        subsample=0.737564575608887,
    )
    xgb_model.fit(train_x,train_y,eval_set=[(val_x,val_y)],early_stopping_rounds=100,verbose=False)
    preds+=xgb_model.predict_proba(test[all_features])[:,1]/kf.n_splits
    oof_predictions += xgb_model.predict_proba(data[all_features])[:,1]/kf.n_splits
    roc.append(roc_auc_score( val_y , xgb_model.predict_proba(val_x)[:,1]))
    print(f"ROC {n+1}: {roc[n]}")
    n=n+1

ROC 1: 0.8977274073470123
ROC 2: 0.8884906464057253
ROC 3: 0.8921360358624577
ROC 4: 0.8930562082119967
ROC 5: 0.8948031615691217
ROC 6: 0.8904214607574558
ROC 7: 0.8951511507427106
ROC 8: 0.8931693095438739
ROC 9: 0.894616858224867
ROC 10: 0.8978565323186428
ROC 11: 0.8936215893300455
ROC 12: 0.8927119738656718
ROC 13: 0.8935649359484674
ROC 14: 0.8968216380126933
ROC 15: 0.8896981714630343
ROC 16: 0.8930864294356909
ROC 17: 0.8988027716149029
ROC 18: 0.8938924847098071
ROC 19: 0.8937420062980209
ROC 20: 0.8949170424008549


In [20]:
sub['target'] = preds
sub.to_csv('xgb_1.csv' , index = False)